In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
np.random.seed(SEED)

data = pd.read_csv('DATA_DESA.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.7 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon = individual
    if C <= 0 or epsilon <= 0 :
        return float('inf'),
    svr = SVR(kernel='linear', C=C, epsilon=epsilon)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
parameter_ranges = [(0.01, 5), (0.001, 5)]

# Register genetic operators
toolbox.register("attr_float", random.uniform)
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (lambda: random.uniform(*parameter_ranges[0]),
                  lambda: random.uniform(*parameter_ranges[1])), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.1)  # Menggunakan crossover uniform
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Menggunakan mutasi flipbit
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 70
population = toolbox.population(n=population_size)

# Genetic algorithm parameters
crossover_rate = 0.9
mutation_rate = 0.1
num_generations = 125

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon = best_individual

# Test the best SVR model
best_svr = SVR(kernel='linear', C=C, epsilon=epsilon)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu: [1.1661763130305307, 2.130067605998172], MAPE: 49.03560405882388
Individu: [2.038203047631326, 2.5943243178882187], MAPE: 49.03560405882388
Individu: [0.629105182082458, 4.074968560392691], MAPE: 49.03560405882388
Individu: [2.6001701551051823, 1.1647507938731942], MAPE: 49.03560405882388
Individu: [1.3093398003336065, 0.9150969331165887], MAPE: 49.03560405882388
Individu: [4.388654059975375, 1.8252549492694066], MAPE: 49.03560405882388
Individu: [3.78429958773485, 0.7957479634412482], MAPE: 49.03560405882388
Individu: [3.0177565760300262, 4.931286290551963], MAPE: 49.03560405882388
Individu: [2.766205694041783, 4.976466052755235], MAPE: 49.03560405882388
Individu: [0.22952014458342843, 0.050809525451812894], MAPE: 47.05345307582419
Individu: [1.85710840171347, 0.1849403547780362], MAPE: 46.75234160957778
Individu: [2.9211776125392848, 3.6896203756019768], MAPE: 49.03560405882388
Individu: [0.9236022321395894, 0.8452826452043933], MAPE: 49.03560405882388
Individu:

In [6]:
# Nilai minimum dan maksimum asli dari data yang digunakan untuk normalisasi
data_min = 0  # Sesuaikan dengan nilai minimum data asli Anda
data_max = 20  # Sesuaikan dengan nilai maksimum data asli Anda

# Denormalisasi secara manual
y_pred_denormalized = y_pred * (data_max - data_min) + data_min

#pred_denormalized =  MinMaxScaler.inverse_transform(y_pred.reshape(1, -1))
print(y_pred_denormalized)

[12.98787782 11.14918345 11.14918345 11.14918345 11.60885704 10.68950986
 13.907225   14.3668986  10.22983626  7.47179471  7.9314683  13.907225
  9.31048908 12.98787782 11.14918345  6.55244752 13.907225   11.60885704]
